# Modes and Separability for the Square Core fiber

Here we find the modes and check to see if they are product functions (ie the equal products of functions only of x and y individually).

In [ ]:
import ngsolve as ng
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.webgui import Draw
from square_fiber import SquareFiber


In [ ]:
A = SquareFiber(ts=[.8e-6, .2e-6, .5e-6], wl=.25e-6,
                maxhs=[.1, .05, .1], mats=['air', 'glass', 'Outer']
               )

In [ ]:
Draw(A.index, A.mesh)

In [ ]:
beta_guess = .999 * A.k * A.ns[0]

In [ ]:
z_guess = A.sqrZfrom(beta_guess)


In [ ]:
z_guess

In [ ]:
z, yr, _, _, _, _ = A.leakymode(p=2, ctr= 2.22253527-0.00748612j, 
                                rad=.01,
#                                 quadrule='ellipse_trapez_shift',
#                                 rhoinv=.99,
                                alpha=0, 
                                npts=4, 
                                nspan=2, 
                                niterations=5,
                                nrestarts=0,
                                stop_tol=1e-8)

In [ ]:
for f in yr:
    Draw(1e-3*f, A.mesh)


# Plot cross sections of core 

In [ ]:
S = np.linspace(-A.Rout, A.Rout, 10001)

In [ ]:
x0, y0 = 0,0
f = yr[0]
xs, ys = [A.mesh(s, y0) for s in S], [A.mesh(x0, s) for s in S]
fxs = np.array([f(xpt) for xpt in xs])
fys = np.array([f(ypt) for ypt in ys])

In [ ]:
%matplotlib inline
plt.plot(S, fxs.real)
# plt.plot(S, fxs.imag)



In [ ]:
plt.plot(S, fys.real)
# plt.plot(S, fys.imag)

That's extremely symmetric

# Let's look at the z component of the vector mode

In [ ]:
A = SquareFiber(ts=[.8e-6, .2e-6, .5e-6], wl=.25e-6,
                maxhs=[.1, .02, .05], mats=['air', 'glass', 'Outer']
               )

In [ ]:
Draw(A.mesh)

In [ ]:
betas, zs, E, phi, _ = A.leakyvecmodes(ctr=7.605, rad=.01, p=2, npts=2, nspan=2, alpha=0,
                                       niterations=4, nrestarts=0, stop_tol=1e-8, sqpml=True,
#                                       rhoinv=.95, quadrule='ellipse_trapez_shift',
                                      )


In [ ]:
for p in phi:
    Draw(1e0*p, A.mesh)

In [ ]:
for e in E:
    Draw(e.real, A.mesh, vectors={'grid_size':500})

In [ ]:
((betas*A.scale)**2).imag

In [ ]:
for beta, p,e in zip(betas, phi, E):
#     beta *= A.scale
#     frac = -1/(2 * ng.Integrate(e.Norm()**2, A.mesh))
#     print(frac*ng.Integrate(beta*ng.grad(p)*ng.Conj(e) + beta.conj()*ng.Conj(ng.grad(p))*e, A.mesh))
    Draw((ng.grad(p)*ng.Conj(e)).imag, A.mesh)

# Even simpler geometry

In [ ]:
A = SquareFiber(ts=[.8e-6,.5e-6], wl=.25e-6,
                maxhs=[.5,  .08], mats=['air', 'Outer']
               )

In [ ]:
Draw(A.mesh)

In [ ]:
betas, zs, E, phi, _ = A.leakyvecmodes(ctr=3.85, rad=.02, p=3, npts=2, nspan=2, alpha=None,
                                       niterations=7, nrestarts=0, stop_tol=1e-8)


In [ ]:
for p in phi:
    Draw(1e0*p, A.mesh)

In [ ]:
for e in E:
    Draw(1e0*e.Norm(), A.mesh)

In [ ]:
((betas*A.scale)**2).imag

In [ ]:
for beta, p,e in zip(betas, phi, E):
    beta *= A.scale
#     frac = -1/(2j * ng.Integrate(e.Norm()**2, A.mesh))
#     print(frac*ng.Integrate(beta*ng.grad(p)*ng.Conj(e) - beta.conj()*ng.Conj(ng.grad(p))*e, A.mesh))
    Draw(1j * beta * ng.grad(p)*ng.Conj(e), A.mesh)

# Cross sections

In [ ]:
x0, y0 = .2,0
F = (E[0]).Norm()
xs, ys = [A.mesh(s, y0) for s in S], [A.mesh(x0, s) for s in S]
Fxs = np.array([F(x) for x in xs], dtype=complex)
Fys = np.array([F(y) for y in ys], dtype=complex)

In [ ]:
%matplotlib inline
plt.plot(S, Fxs.real)
# plt.plot(S, Fxs.imag)


In [ ]:
plt.plot(S, Fys.real)
plt.plot(S, Fys.imag)

# Create product function

To see if the above really are product functions, we should be able to recreate them using the two cross sections we have

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
u_0 = F(A.mesh(x0,y0))

In [ ]:
u_0

In [ ]:
X, Y = np.meshgrid(S,S)
Z = np.outer(Fxs, Fys)

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis');
# ax.plot_surface(X, Y, Z.imag, cmap='viridis')

In [ ]:
Z_true = np.zeros_like(X, dtype=complex)

In [ ]:
for i,x in enumerate(X):
    for j, y in enumerate(Y):
        Z_true[i,j] = F(A.mesh(x[i],y[j]))

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z_true.real, cmap='viridis')
ax.plot_surface(X, Y, Z.real, cmap='turbo')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, (Z-Z_true).real, cmap='viridis')

# Does seem likely that this is a product function

We are having troubles getting it in the corners, but this isn't necessarily because it's not a product but because corners are hard to capture using finite elements.

It's also good that the imaginary part of the ratio of the two is near machine zero.

# Improve the finite element approach

If we made our outer boundary square, we expect that we know the radial derivative, it would be related to exponential decay and our eigenvalue (in some way).  We could try to implement that, or also we could just do Neumann conditions on a square outer boundary far from the core.  We could also use CSG 2D and up the precision at the corners using point info and decreasing the maxh there.  

All worth doing, but also it seems likely that we can go ahead theoretically and try to figure out the product function theory.  We'd have to line up regions etc.  We could try to get a sense of what the constants of separation are on each region using the numerics from this notebook (especially after implementing some of the above fixes).